<a href="https://colab.research.google.com/github/SELOMELO280104/Roboflow_test/blob/main/bounding_box.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bounding Box Test


In [ ]:
# Gerekli kütüphaneleri kur
!pip install ultralytics -q
import os
import zipfile
from google.colab import files

# 1. Zip dosyasını yükleme ekranı
print("👇 Lütfen Roboflow'dan indirdiğiniz Bounding Box '.zip' veri setini seçin:")
uploaded = files.upload()

if uploaded:
    zip_filename = list(uploaded.keys())[0]
    extract_dir = '/content/dataset_bbox'

    # 2. Zip dosyasını çıkartma
    print(f"\n📦 '{zip_filename}' çıkartılıyor...")
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    # 3. data.yaml dosyasını otomatik bulma
    yaml_path = None
    for root, dirs, dosyalar in os.walk(extract_dir):
        if 'data.yaml' in dosyalar:
            yaml_path = os.path.join(root, 'data.yaml')
            break

    if yaml_path:
        print(f"✅ Veri seti hazır! Dosya yolu: {yaml_path}")
    else:
        print("❌ HATA: Zip dosyasının içinde 'data.yaml' bulunamadı!")
else:
    print("❌ Dosya yüklemesi iptal edildi.")

In [ ]:
from ultralytics import YOLO

# Veri seti yolunu tanımla (1. hücreden gelen yol)
dataset_yaml = '/content/dataset_bbox/data.yaml' # Zip farklı çıkarsa burayı yaml_path olarak da verebiliriz.

print("\n🚀 Bounding Box (Kutu) eğitimi T4 GPU üzerinde başlıyor...")
model = YOLO('yolov8n.pt')

results = model.train(
    data=dataset_yaml,
    epochs=30,
    imgsz=640,
    batch=16,
    project='/content/Trained_model',
    name='ayakkabi_bbox_sonuc',
    exist_ok=True
)
print("\n🎉 Eğitim başarıyla tamamlandı!")

In [ ]:
import glob
from IPython.display import Image, display
from ultralytics import YOLO

# 1. Eğittiğin Bounding Box modelini yükle
best_model_path = '/content/Trained_model/ayakkabi_bbox_sonuc/weights/best.pt'
model = YOLO(best_model_path)

# 2. Test resimlerinin olduğu klasör
test_klasoru = '/content/dataset_bbox/test/images'

print("🤖 Test klasöründeki resimler için tahmin yapılıyor...")

# 3. Tahmin yap ve sonuçları kaydet
sonuclar = model.predict(
    source=test_klasoru,
    conf=0.03,          # Güven eşiği
    save=True,
    project='/content/Trained_model',
    name='tum_test_tahminleri_bbox',
    exist_ok=True
)

# 4. İşlenip kutu çizilen resimleri bul
cizilen_resimler = glob.glob('/content/Trained_model/tum_test_tahminleri_bbox/*.jpg')

print(f"\n✅ Toplam {len(cizilen_resimler)} resim işlendi. İşte sonuçlar:")

# 5. Resimleri alt alta ekrana bas
for img_path in cizilen_resimler:
    display(Image(filename=img_path))

In [ ]:
from google.colab import files

best_model_path = '/content/Trained_model/ayakkabi_bbox_sonuc/weights/best.pt'
print("⬇️ Eğitilmiş Bounding Box modeli (best.pt) bilgisayarınıza indiriliyor...")

try:
    files.download(best_model_path)
    print("✅ İndirme işlemi başlatıldı!")
except Exception as e:
    print(f"❌ İndirme sırasında bir hata oluştu: {e}\nLütfen sol menüden manuel olarak indirin.")